# Estimate AVO response to calibrate data

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

from IPython.display import HTML

import seppy
import os

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def plotseis(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=0, xmax=650, grid=True, colorbar=False, curve=np.array([]), linestyle='-', animation=False):
    """Plot 2D seismic image"""
 
    vmin=np.min(d) / p
    vmax=p*np.max(d)
    
    if p<0:
        vmax=-p*np.amax(d)
        vmin=-vmax
        
    if valmin<valmax:
        vmin=valmin
        vmax=valmax
        
    nx=d.shape[0]
                
    if animation==False:
        fig=plt.figure(figsize=(6, 4),dpi=100)
        
    im = plt.imshow(np.transpose(d),interpolation='sinc',aspect="auto",extent=[xmin,xmax,tmax,tmin],cmap='Greys',vmin=vmin,vmax=vmax)
    plt.xlabel('Offset (m)')
    plt.ylabel('Time (s)')
    if grid==True:
        plt.grid(color='r', linestyle='-', linewidth=0.2)
    if colorbar==True:
        plt.colorbar()
    if curve.size>0:
        plt.plot(np.linspace(xmin,xmax,nx),curve,linestyle)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    
    if animation==False:
        plt.show()
    else:
        return im

def plotanim(d, p=0.9, valmin=1, valmax=-1, tmin=0, tmax=0.35, xmin=0, xmax=650, grid=True, colorbar=False):
    """generate list of figures for animation"""

    assert(d.ndim==3)

    fig=plt.figure(figsize=(6, 4),dpi=100)
    ims=[]
    n=d.shape[0]
    for i in range(n):
        im = plotseis(d[i], p=p, valmin=valmin, valmax=valmax, tmin=tmin, tmax=tmax, xmin=xmin, xmax=xmax, grid=grid, colorbar=colorbar, animation=True)
        ims.append([im])

    animation = anim.ArtistAnimation(fig, ims, interval=500, blit=True,repeat=True, repeat_delay=0)
    plt.close()

    return animation

In [ ]:
# read data, synthetic, and mask

sep = seppy.sep()

data_axes, data = sep.read_file(datapath+"ch5_data_group4.H.bp")
pdata = data.reshape(data_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_syn_group4.H.bp")
psyn = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_syn_3d_sp49.H.hankel.bp")
psyn3d = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch5_mute_group4.H.select")
pmute = data.reshape(axes.n,order='F').T

nt=data_axes.n[0]
ntr=data_axes.n[1]
ns=data_axes.n[2]
dt=data_axes.d[0]
dx=data_axes.d[1]

In [ ]:
animation1 = plotanim(pdata,p=0.9, valmin=-1, valmax=1, tmin=0, tmax=0.35, xmin=-650, xmax=650, grid=True, colorbar=False)
animation2 = plotanim(psyn,p=0.9, valmin=-1, valmax=1, tmin=0, tmax=0.35, xmin=-650, xmax=650, grid=True, colorbar=False)

In [ ]:
HTML(animation1.to_jshtml())

In [ ]:
HTML(animation2.to_jshtml())

In [ ]:
# select the shots to be dropped in the summation
select = [0,1,2,3,9,13]

# stack data to create macro shot gather
macro_data=np.sum(np.delete(pdata,select,axis=0)*np.delete(pmute,select,axis=0),axis=0)
macro_syn=np.sum(np.delete(psyn,select,axis=0)*np.delete(pmute,select,axis=0),axis=0)

macro_mask=np.sum(pmute,axis=0) + 0.01
macro_data/=macro_mask
macro_syn/=macro_mask

# compute the envelop
macro_data_env = np.absolute(signal.hilbert(macro_data,axis=1))
macro_syn_env = np.absolute(signal.hilbert(macro_syn,axis=1))

# split negative and positive offsets for unstimulated and stimulated reservoir
nhalf = int((ntr-1)/2)
data_n = np.flip(macro_data_env[:nhalf+1,:],axis=0)
syn_n = np.flip(macro_syn_env[:nhalf+1,:],axis=0)
data_p=macro_data_env[nhalf:,:]
syn_p=macro_syn_env[nhalf:,:]

# keep |offset| 200 to 450 m 
offmin=200
offmax=450
nx=offmax-offmin+1
data_n = data_n[offmin:offmax+1,:]
data_p = data_p[offmin:offmax+1,:]
syn_n = syn_n[offmin:offmax+1,:]
syn_p = syn_p[offmin:offmax+1,:]

In [ ]:
# pick the P-maximum and the S-maximum
vel_p=4300.
vel_s=1700.
twidth_p=6
twidth_s=6
t0_p=0.016
t0_s=0.03

data_n_max=np.zeros((nx,1))
data_n_amax=np.zeros((nx,1),dtype=int)
syn_n_max=np.zeros((nx,1))
syn_n_amax=np.zeros((nx,1),dtype=int)

data_p_max=np.zeros((nx,2))
data_p_amax=np.zeros((nx,2),dtype=int)
syn_p_max=np.zeros((nx,2))
syn_p_amax=np.zeros((nx,2),dtype=int)

for i in range(nx):
    offset = i*dx + offmin
    t_p = offset/vel_p
    t_s = offset/vel_s
    it0_p = int(t0_p/dt)
    it0_s = int(t0_s/dt)
    it_p = int(t_p/dt) + it0_p
    it_s = int(t_s/dt) + it0_s
    syn_n_amax[i,0] = it_p-twidth_p + int(np.argmax(syn_n[i,it_p-twidth_p:it_p+twidth_p]))
    data_n_amax[i,0] = it_p-twidth_p + int(np.argmax(data_n[i,it_p-twidth_p:it_p+twidth_p]))
    syn_p_amax[i,0] = it_p-twidth_p + int(np.argmax(syn_p[i,it_p-twidth_p:it_p+twidth_p]))
    data_p_amax[i,0] = it_p-twidth_p + int(np.argmax(data_p[i,it_p-twidth_p:it_p+twidth_p]))
    syn_p_amax[i,1] = it_s-twidth_s + int(np.argmax(syn_p[i,it_s-twidth_s:it_s+twidth_s]))
    data_p_amax[i,1] = it_s-twidth_s + int(np.argmax(data_p[i,it_s-twidth_s:it_s+twidth_s]))

    syn_n_max[i,0] = syn_n[i,syn_n_amax[i,0]]
    data_n_max[i,0] = data_n[i,data_n_amax[i,0]]
    syn_p_max[i,0] = syn_p[i,syn_p_amax[i,0]]
    data_p_max[i,0] = data_p[i,data_p_amax[i,0]]
    syn_p_max[i,1] = syn_p[i,syn_p_amax[i,1]]
    data_p_max[i,1] = data_p[i,data_p_amax[i,1]]

In [ ]:
plotseis(syn_p,valmin=-1,valmax=1, xmin=offmin, xmax=offmax, curve=dt*syn_p_amax)
plotseis(data_p,valmin=-1,valmax=1, xmin=offmin, xmax=offmax, curve=dt*data_p_amax)

In [ ]:
# Fit lines to the AVO curves
data_n_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(data_n_max[:,0]),deg=1)
syn_n_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn_n_max[:,0]),deg=1)
data_p_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(data_p_max[:,0]),deg=1)
syn_p_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn_p_max[:,0]),deg=1)
data_p_fit_s = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(data_p_max[:,1]),deg=1)
syn_p_fit_s = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn_p_max[:,1]),deg=1)

print("The fitted line for P negative offset data has a slope p = %f" %data_n_fit_p[0])
print("The fitted line for P negative offset synthetic has a slope q = %f" %syn_n_fit_p[0])
print("The fitted line for P positive offset data has a slope p = %f" %data_p_fit_p[0])
print("The fitted line for P positive offset synthetic has a slope q = %f" %syn_p_fit_p[0])
print("The fitted line for S positive offset data has a slope p = %f" %data_p_fit_s[0])
print("The fitted line for S positive offset synthetic has a slope q = %f" %syn_p_fit_s[0])

In [ ]:
plt.figure(figsize=(12, 4))

data_array=data_n_max[:,0]
syn_array=syn_n_max[:,0]
p=data_n_fit_p
q=syn_n_fit_p

plt.scatter(np.log10(np.linspace(offmin,offmax,nx)),np.log10(data_array),color='b',s=1,label="Max envelop - observed")
plt.scatter(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn_array),color='r',s=1,label="Max envelop - modeled")
plt.plot(np.log10(np.linspace(offmin,offmax,nx)),np.log10(np.linspace(offmin,offmax,nx))*p[0]+p[1],'b--',linewidth=2,label="Fitted line - observed")
plt.plot(np.log10(np.linspace(offmin,offmax,nx)),np.log10(np.linspace(offmin,offmax,nx))*q[0]+q[1],'r--',linewidth=2,label="Fitted line - modeled")
plt.xlabel(r"$\log_{10}$(|Offset|)",fontsize=12)
plt.ylabel(r"$\log_{10}$(Amplitude)",fontsize=12)
plt.legend()
plt.show()

#### Compute weighting function
Based on amplitude decay vs offset obtained from the envelop of field and synthetic super shots, a weighting function is deduced, to be applied to field data before FWI
$$f(x) = (\frac{x}{x_0})^{q-p}$$
where $x=\text{offset}$, and $x_0=200 \text{ m}$.
The estimation of $p$ and $q$ for P- and S-arrivals is very uncertain and sensitive to how the super shots are built. I rounded off the values such that $q-p\approx 1$.

In [ ]:
# Compute AVO correction weights
q_m_p = 1.

offsets=np.linspace(-800,800,1601)
weights=(np.abs(offsets)/offmin)**(q_m_p)

In [ ]:
# save the weights
sep.write_file(datapath+"ch5_avo_weights.H", np.transpose(weights), ds=np.array([dx]), os=np.array([-800]), dpath=datapath)

### Compare 2D and 3D synthetic AVO

In [ ]:
# compute the envelop
sp=11
syn2d_env = np.absolute(signal.hilbert(psyn[sp],axis=1))
syn3d_env = np.absolute(signal.hilbert(psyn3d,axis=1))

# split negative and positive offsets for unstimulated and stimulated reservoir
nhalf = int((ntr-1)/2)
syn2d_n = np.flip(syn2d_env[:nhalf+1,:],axis=0)
syn3d_n = np.flip(syn3d_env[:nhalf+1,:],axis=0)
syn2d_p=syn2d_env[nhalf:,:]
syn3d_p=syn3d_env[nhalf:,:]

# keep |offset| 200 to 450 m 
offmin=200
offmax=450
nx=offmax-offmin+1
syn2d_n = syn2d_n[offmin:offmax+1,:]
syn2d_p = syn2d_p[offmin:offmax+1,:]
syn3d_n = syn3d_n[offmin:offmax+1,:]
syn3d_p = syn3d_p[offmin:offmax+1,:]

In [ ]:
# pick the P-maximum and the S-maximum
vel_p=4300.
vel_s=2000.
twidth_p=6
twidth_s=6
t0_p=0.016
t0_s=0.05

syn2d_n_max=np.zeros((nx,1))
syn2d_n_amax=np.zeros((nx,1),dtype=int)
syn3d_n_max=np.zeros((nx,1))
syn3d_n_amax=np.zeros((nx,1),dtype=int)

syn2d_p_max=np.zeros((nx,2))
syn2d_p_amax=np.zeros((nx,2),dtype=int)
syn3d_p_max=np.zeros((nx,2))
syn3d_p_amax=np.zeros((nx,2),dtype=int)

for i in range(nx):
    offset = i*dx + offmin
    t_p = offset/vel_p
    t_s = offset/vel_s
    it0_p = int(t0_p/dt)
    it0_s = int(t0_s/dt)
    it_p = int(t_p/dt) + it0_p
    it_s = int(t_s/dt) + it0_s
    syn2d_n_amax[i,0] = it_p-twidth_p + int(np.argmax(syn2d_n[i,it_p-twidth_p:it_p+twidth_p]))
    syn3d_n_amax[i,0] = it_p-twidth_p + int(np.argmax(syn3d_n[i,it_p-twidth_p:it_p+twidth_p]))
    syn2d_p_amax[i,0] = it_p-twidth_p + int(np.argmax(syn2d_p[i,it_p-twidth_p:it_p+twidth_p]))
    syn3d_p_amax[i,0] = it_p-twidth_p + int(np.argmax(syn3d_p[i,it_p-twidth_p:it_p+twidth_p]))
    syn2d_p_amax[i,1] = it_s-twidth_s + int(np.argmax(syn2d_p[i,it_s-twidth_s:it_s+twidth_s]))
    syn3d_p_amax[i,1] = it_s-twidth_s + int(np.argmax(syn3d_p[i,it_s-twidth_s:it_s+twidth_s]))

    syn2d_n_max[i,0] = syn2d_n[i,syn2d_n_amax[i,0]]
    syn3d_n_max[i,0] = syn3d_n[i,syn3d_n_amax[i,0]]
    syn2d_p_max[i,0] = syn2d_p[i,syn2d_p_amax[i,0]]
    syn3d_p_max[i,0] = syn3d_p[i,syn3d_p_amax[i,0]]
    syn2d_p_max[i,1] = syn2d_p[i,syn2d_p_amax[i,1]]
    syn3d_p_max[i,1] = syn3d_p[i,syn3d_p_amax[i,1]]

In [ ]:
plotseis(syn2d_p,valmin=-1,valmax=1, xmin=offmin, xmax=offmax, curve=dt*syn2d_p_amax)
plotseis(syn3d_p,valmin=-1,valmax=1, xmin=offmin, xmax=offmax, curve=dt*syn3d_p_amax)

In [ ]:
# Fit lines to the AVO curves
syn2d_n_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn2d_n_max[:,0]),deg=1)
syn3d_n_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn3d_n_max[:,0]),deg=1)
syn2d_p_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn2d_p_max[:,0]),deg=1)
syn3d_p_fit_p = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn3d_p_max[:,0]),deg=1)
syn2d_p_fit_s = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn2d_p_max[:,1]),deg=1)
syn3d_p_fit_s = np.polyfit(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn3d_p_max[:,1]),deg=1)

print("The fitted line for P negative offset 2d has a slope p = %f" %syn2d_n_fit_p[0])
print("The fitted line for P negative offset 3d has a slope q = %f" %syn3d_n_fit_p[0])
print("The fitted line for P positive offset 2d has a slope p = %f" %syn2d_p_fit_p[0])
print("The fitted line for P positive offset 3d has a slope q = %f" %syn3d_p_fit_p[0])
print("The fitted line for S positive offset 2d has a slope p = %f" %syn2d_p_fit_s[0])
print("The fitted line for S positive offset 3d has a slope q = %f" %syn3d_p_fit_s[0])

In [ ]:
plt.figure(figsize=(12, 4))

data_array=syn2d_p_max[:,0]
syn_array=syn3d_p_max[:,0]
p=syn2d_p_fit_p
q=syn3d_p_fit_p

plt.scatter(np.log10(np.linspace(offmin,offmax,nx)),np.log10(data_array),color='b',s=1,label="Max envelop - 2d")
plt.scatter(np.log10(np.linspace(offmin,offmax,nx)),np.log10(syn_array),color='r',s=1,label="Max envelop - 3d")
plt.plot(np.log10(np.linspace(offmin,offmax,nx)),np.log10(np.linspace(offmin,offmax,nx))*p[0]+p[1],'b--',linewidth=2,label="Fitted line - 2d")
plt.plot(np.log10(np.linspace(offmin,offmax,nx)),np.log10(np.linspace(offmin,offmax,nx))*q[0]+q[1],'r--',linewidth=2,label="Fitted line - 3d")
plt.xlabel(r"$\log_{10}$(|Offset|)",fontsize=12)
plt.ylabel(r"$\log_{10}$(Amplitude)",fontsize=12)
plt.legend()
plt.show()

#### Estimate Q for P and S waves
The amplitude function can be written as
$$A(x)=A(x_0)(\frac{x}{x_0})^q \exp(\frac{-\pi f(x-x_0)}{cQ})$$
where $x_0$ is the reference offset, $q$ is the power describing the geometrical spreading, $f$ here is the frequency. So strictly speaking, I cannot fit a straight line to $\log(A)$ vs $\log(x)$ unless $cQ>>\pi f(x-x_0)$. Now assuming that the geometrical spreading can a known $q$, I can scan $Q$ to best fit with the picked amplitudes. I also want to allow bulk shifts of the amplitudes because relying on a fixed $A(x_0)$ may not be robust.
I set $c=4300$ or $1700$ m/s, $x_0=200$ m, and $f=100$ Hz. For a perfect waveguide (cylindrical spreading), $q=-0.5$. For spherical spreading, $q=-1$. 

In [ ]:
def amplitude(x,Q,A0,x0=200,q=-0.5,f=80,c=4300):
    """Compute the AVO curve"""
    A = A0 * (x0/x)**(-q)*np.exp(-np.pi *f * (x-x0) / (c*Q))
    return A

def functional(A_true, x, Q, A0, x0=200,q=-0.5,f=80,c=4300):
    """Perform a grid search over Q and A0, compute the L2 residual norm, and find the global minimum"""
    nQ = Q.shape[0]
    nA = A0.shape[0]
    func = np.zeros((nA,nQ))
    A0_opt = 0
    Q_opt=0
    func_opt=999999.
    for ia in range(nA):
        for iq in range(nQ):
            A = amplitude(x,Q[iq],A0[ia],x0,q,f,c)
            func[ia,iq] = np.linalg.norm(A-A_true)
            if func[ia,iq]<func_opt:
                func_opt = func[ia,iq]
                A0_opt = A0[ia]
                Q_opt = Q[iq]
    return func, Q_opt, A0_opt

In [ ]:
A_true=data_n_max[:,0]
x = np.linspace(offmin,offmax,nx)
Q = np.linspace(10,200,20)
A0 = np.linspace(0.8,1.2,9)*A_true[0]
extent = [Q[0]-5,Q[-1]-5,A0[-1]-0.025,A0[0]-0.025]
x0 = offmin
f = 100.
c = 4300.
q05 = -0.5
q1= -1.
q15= -1.5

func_n_p_q05, Q_opt_n_p_q05, A0_opt_n_p_q05 = functional(A_true,x,Q,A0,x0=x0,q=q05,f=f,c=c)
A_fit_q05 = amplitude(x,Q_opt_n_p_q05,A0_opt_n_p_q05,x0=x0,q=q05,f=f,c=c)

func_n_p_q1, Q_opt_n_p_q1, A0_opt_n_p_q1 = functional(A_true,x,Q,A0,x0=x0,q=q1,f=f,c=c)
A_fit_q1 = amplitude(x,Q_opt_n_p_q1,A0_opt_n_p_q1,x0=x0,q=q1,f=f,c=c)

func_n_p_q15, Q_opt_n_p_q15, A0_opt_n_p_q15 = functional(A_true,x,Q,A0,x0=x0,q=q15,f=f,c=c)
A_fit_q15 = amplitude(x,Q_opt_n_p_q15,A0_opt_n_p_q15,x0=x0,q=q15,f=f,c=c)

In [ ]:
fig=plt.figure(figsize=(6.66, 6),dpi=300)

plt.subplot(2,2,1)
plt.imshow(func_n_p_q05,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel(r'$Q_P$ factor')
plt.ylabel(r'$A_0$ (scaled strain rate)')
cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_n_p_q05,A0_opt_n_p_q05,c='w',s=50)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"a) $q = -0.5$",loc='center')

plt.subplot(2,2,2)
plt.imshow(func_n_p_q1,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel(r'$Q_P$ factor')
plt.ylabel(r'$A_0$ (scaled strain rate)')
# plt.gca().set_yticklabels([])
cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_n_p_q1,A0_opt_n_p_q1,c='w',s=50)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"b) $q = -1.0$",loc='center')

plt.subplot(2,2,3)
plt.imshow(func_n_p_q15,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel(r'$Q_P$ factor')
plt.ylabel(r'$A_0$ (scaled strain rate)')
cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_n_p_q15-5,A0_opt_n_p_q15-0.025,c='w',s=50, clip_on=False)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"c) $q = -1.5$",loc='center')

plt.subplot(2,2,4)
plt.scatter(x,A_true,color='k',s=1,label="Max envelop")
plt.plot(x,A_fit_q05,'b-',linewidth=1,label=r"Fitted line, $q=-0.5$")
plt.plot(x,A_fit_q1,'r-',linewidth=1,label=r"Fitted line, $q=-1$")
plt.plot(x,A_fit_q15,'g-',linewidth=1,label=r"Fitted line, $q=-1.5$")
plt.xlabel('Offset (m)')
plt.ylabel(r'$A(x)$ (scaled strain rate)')
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r"d)",loc='center')

plt.tight_layout()

In [ ]:
A_true=data_p_max[:,0]
x = np.linspace(offmin,offmax,nx)
Q = np.linspace(10,500,50)
A0 = np.linspace(0.8,1.2,9)*A_true[0]
extent = [Q[0]-5,Q[-1]-5,A0[-1]-0.025,A0[0]-0.025]
x0 = offmin
f = 100.
c = 4300.
q05 = -0.5
q1= -1.

func_p_p_q05, Q_opt_p_p_q05, A0_opt_p_p_q05 = functional(A_true,x,Q,A0,x0=x0,q=q05,f=f,c=c)
A_fit_q05 = amplitude(x,Q_opt_p_p_q05,A0_opt_p_p_q05,x0=x0,q=q05,f=f,c=c)

func_p_p_q1, Q_opt_p_p_q1, A0_opt_p_p_q1 = functional(A_true,x,Q,A0,x0=x0,q=q1,f=f,c=c)
A_fit_q1 = amplitude(x,Q_opt_p_p_q1,A0_opt_p_p_q1,x0=x0,q=q1,f=f,c=c)

In [ ]:
fig=plt.figure(figsize=(6.66, 6),dpi=300)

plt.subplot(2,2,1)
plt.imshow(func_p_p_q05,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel('Q factor')
plt.ylabel(r'$A_0$ (scaled strain rate)')
cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
cbar.ax.tick_params(labelsize=6)
# cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_p_p_q05,A0_opt_p_p_q05,c='w',s=50)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"a) $q = -0.5$",loc='center')

plt.subplot(2,2,2)
plt.imshow(func_p_p_q1,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel('Q factor')
plt.gca().set_yticklabels([])
cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
cbar.ax.tick_params(labelsize=6)
cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_p_p_q1,A0_opt_p_p_q1,c='w',s=50)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"b) $q = -1.0$",loc='center')

plt.subplot(2,2,4)
plt.scatter(x,A_true,color='k',s=1,label="Max envelop")
plt.plot(x,A_fit_q05,'b-',linewidth=2,label=r"Fitted line, $q=-0.5$")
plt.plot(x,A_fit_q1,'r-',linewidth=2,label=r"Fitted line, $q=-1$")
plt.xlabel('Offset (m)')
plt.ylabel(r'$A(x)$ (scaled strain rate)')
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r"c)",loc='center')

plt.tight_layout()

In [ ]:
A_true=data_p_max[:,1]
x = np.linspace(offmin,offmax,nx)
Q = np.linspace(10,250,20)
A0 = np.linspace(0.8,1.2,9)*A_true[0]
extent = [Q[0]-5,Q[-1]-5,A0[-1]-0.025,A0[0]-0.025]
x0 = offmin
f = 80.
c = 1700.
q05 = -0.5
q1= -1.
q15= -1.5

func_p_s_q05, Q_opt_p_s_q05, A0_opt_p_s_q05 = functional(A_true,x,Q,A0,x0=x0,q=q05,f=f,c=c)
A_fit_q05 = amplitude(x,Q_opt_p_s_q05,A0_opt_p_s_q05,x0=x0,q=q05,f=f,c=c)

func_p_s_q1, Q_opt_p_s_q1, A0_opt_p_s_q1 = functional(A_true,x,Q,A0,x0=x0,q=q1,f=f,c=c)
A_fit_q1 = amplitude(x,Q_opt_p_s_q1,A0_opt_p_s_q1,x0=x0,q=q1,f=f,c=c)

func_p_s_q15, Q_opt_p_s_q15, A0_opt_p_s_q15 = functional(A_true,x,Q,A0,x0=x0,q=q15,f=f,c=c)
A_fit_q15 = amplitude(x,Q_opt_p_s_q15,A0_opt_p_s_q15,x0=x0,q=q15,f=f,c=c)

In [ ]:
fig=plt.figure(figsize=(6.66, 6.66),dpi=300)

plt.subplot(2,2,1)
plt.imshow(func_p_s_q05,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel(r'$Q_S$')
plt.ylabel(r'$A_0$ (scaled strain rate)')
# cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
# cbar.ax.tick_params(labelsize=6)
# cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_p_s_q05,A0_opt_p_s_q05,c='r',s=50, clip_on=False)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"a) $q = -0.5$",loc='left')

plt.subplot(2,2,2)
plt.imshow(func_p_s_q1,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel(r'$Q_S$')
# plt.ylabel(r'$A_0$')
plt.gca().set_yticklabels([])
# cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
# cbar.ax.tick_params(labelsize=6)
# cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_p_s_q1-5,A0_opt_p_s_q1-0.025,c='m',s=50, clip_on=False)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"b) $q = -1.0$",loc='left')

plt.subplot(2,2,3)
plt.imshow(func_p_s_q15,aspect='auto', interpolation='spline36', extent=extent, cmap='jet')
plt.xlabel(r'$Q_S$')
plt.ylabel(r'$A_0$ (scaled strain rate)')
# cbar = plt.colorbar(shrink=0.7, format="%.0f",pad=0.02)
# cbar.ax.tick_params(labelsize=6)
# cbar.set_label(label="Residual L2 norm",size=6)
plt.scatter(Q_opt_p_s_q15-5,A0_opt_p_s_q15-0.025,c='g',s=50, clip_on=False)
plt.gca().invert_yaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.grid(color='w', linestyle='-', linewidth=0.2)
plt.title(r"c) $q = -1.5$",loc='left')

plt.subplot(2,2,4)
plt.scatter(x,A_true,color='k',s=1,label="Max envelop")
plt.plot(x,A_fit_q05,'r-',linewidth=1,label=r"Fitted line, $q=-0.5$")
plt.plot(x,A_fit_q1,'m-',linewidth=1,label=r"Fitted line, $q=-1$")
plt.plot(x,A_fit_q15,'g-',linewidth=1,label=r"Fitted line, $q=-1.5$")
plt.xlabel('Offset (m)')
plt.ylabel(r'$A(x)$ (scaled strain rate)')
plt.legend(loc='upper right',prop={'size': 6})
plt.title(r"d)",loc='left')

plt.tight_layout()

plt.savefig(figpath+'avo_S_wave.png',bbox_inches='tight',format='png')

In [ ]:
print("Qp for negative offsets %.0f %.0f" %(Q_opt_n_p_q05,Q_opt_n_p_q1))
print("Qp for positive offsets %.0f %.0f" %(Q_opt_p_p_q05,Q_opt_p_p_q1))
print("Qs for positive offsets %.0f %.0f" %(Q_opt_p_s_q05,Q_opt_p_s_q1))